## Stocks Filter and Pre Analysis (v2)

This scripts gets data from the websites filtering out stocks/company tickers according the parameters.

In [1]:
# Filter Parameters
MIN_LIQUIDEZ_DIARIA_MEDIA     = 500_000
MIN_FUNDAMENTEI_USERS_SCORE   = 4
MIN_FUNDAMENTEI_USERS_REVIEWS = 50
MAX_YEARS_FOR_HIST_DATA       = 10

------

In [ ]:
import numpy
import os
import pandas
import sys

from tenacity import retry, wait_random, stop_after_attempt
from sklearn.linear_model import LinearRegression  # scikit-learn
from tqdm import tqdm

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.scrapers.statusinvest import StockListScraper as StatusInvestList
from libs.scrapers.statusinvest import StockExtraInfosScraper as StatusInvestExtraInfos
from libs.scrapers.statusinvest import StockHistIndicatorsScraper as StatusInvestHistInfos
from libs.scrapers.fundamentei import StockInfosScraper as FundamenteiInfos


------

In [3]:
# General data from Status Invest
stocks_general_data_df = StatusInvestList().dataframe_format

In [4]:
# Filter out by filters
if MIN_LIQUIDEZ_DIARIA_MEDIA:
    stocks_general_data_df = stocks_general_data_df[
        stocks_general_data_df["LIQUIDEZ MEDIA DIARIA"] >= MIN_LIQUIDEZ_DIARIA_MEDIA
    ]

# Filter out only the less liquid stock from each company
stocks_general_data_df["PREFIX"] = stocks_general_data_df["TICKER"].str.upper().str[:4]
stocks_general_data_df.sort_values("LIQUIDEZ MEDIA DIARIA", ascending=False, inplace=True)
stocks_general_data_df.drop_duplicates("PREFIX", keep="first", inplace=True)
stocks_general_data_df.drop("PREFIX", axis=1, inplace=True)

------

In [5]:
@retry(stop=stop_after_attempt(5), wait=wait_random(min=1, max=5))
def get_stock_infos_df(stock_ticker: str) -> pandas.DataFrame:
    return FundamenteiInfos(stock_ticker).dataframe_format

In [6]:
# get info from all stocks
stock_infos_df = None
failed_tickers = []

for stock_ticker in tqdm(stocks_general_data_df["TICKER"]):
    try:
        # read stock infos
        si_df = get_stock_infos_df(stock_ticker)
        si_df.columns = [stock_ticker]
        si_df = si_df.T
        stock_infos_df = pandas.concat([stock_infos_df, si_df])
    except:
        failed_tickers.append(stock_ticker)

# show failures (tickers)
print(f"Filed stock tickers: {failed_tickers}")

100%|██████████| 226/226 [04:31<00:00,  1.20s/it]

Filed stock tickers: ['BOAS3', 'SQIA3', 'ENBR3', 'MODL3', 'PARD3', 'BRPR3']


In [7]:
# Convet types
stock_infos_df["Nota dos Usuários"] = stock_infos_df["Nota dos Usuários"].astype(float)
stock_infos_df["Num. de Votos"] = stock_infos_df["Num. de Votos"].astype(int)

In [8]:
# Filter out by filters
if MIN_FUNDAMENTEI_USERS_SCORE:
    stock_infos_df = stock_infos_df[
        stock_infos_df['Nota dos Usuários'] >= MIN_FUNDAMENTEI_USERS_SCORE
    ]

if MIN_FUNDAMENTEI_USERS_REVIEWS:
    stock_infos_df = stock_infos_df[
        stock_infos_df['Num. de Votos'] >= MIN_FUNDAMENTEI_USERS_REVIEWS
    ]

-------

In [9]:
# Get extra infos from Status Invest
extra_infos_df = None

for stock_ticker in tqdm(stock_infos_df["Ticker"]):
    ei_df = StatusInvestExtraInfos(stock_ticker).dataframe_format
    ei_df.columns = [stock_ticker]
    ei_df = ei_df.T
    extra_infos_df = pandas.concat([extra_infos_df, ei_df])

100%|██████████| 42/42 [00:56<00:00,  1.35s/it]


-----

In [10]:
# Concatenate all historical data available in the website (API)
def get_historical_data(stock_ticker: str) -> pandas.DataFrame:
    hist_df = None

    # Infos: dy, p_l, p_vp, p_ebita, p_ebit, p_sr, p_ativo, p_capitlgiro,
    #        p_ativocirculante, ev_ebitda, ev_ebit, lpa, vpa, peg_Ratio,
    #        dividaliquida_patrimonioliquido, dividaliquida_ebitda,
    #        dividaliquida_ebit, patrimonio_ativo, passivo_ativo,
    #        liquidezcorrente, margembruta, margemebitda, margemebit,
    #        margemliquida, roe, roa, roic, giro_ativos, receitas_cagr5,
    #        lucros_cagr5
    hist_inds = StatusInvestHistInfos(stock_ticker).dataframe_format
    hist_df = pandas.concat([hist_df, hist_inds])

    # sort columns
    hist_df = hist_df[sorted(hist_df.columns)]

    # return hisotical indicators
    return hist_df

In [11]:
# Linear Regression
def get_linear_regression_trend(series: pandas.Series) -> float:
    # reshape
    x = numpy.array(series.index).reshape((-1, 1))
    y = numpy.array(series.values).reshape((-1, 1))

    # calculate linear regression slope
    model = LinearRegression()
    model.fit(x, y)

    # LR
    # model.score(x, y)  # R
    # model.intercept_[0]  # Intercept
    # model.coef_[0][0]  # Slope

    # predict
    last_year = series.index.max()
    predict_x = numpy.array([last_year, last_year + 1]).reshape((-1, 1))
    predict_result = model.predict(predict_x)

    # trend in %
    return predict_result[1] / predict_result[0] - 1

In [12]:
for stock_ticker in tqdm(stock_infos_df["Ticker"]):
    hd_df = get_historical_data(stock_ticker).T

    # limit historical data
    if MAX_YEARS_FOR_HIST_DATA:
        hd_df = hd_df[-MAX_YEARS_FOR_HIST_DATA:]

    # get min and max years
    stock_infos_df.loc[
        stock_infos_df.index == stock_ticker,
        "Min_HY"
    ] = min(hd_df.index)

    stock_infos_df.loc[
        stock_infos_df.index == stock_ticker,
        "Max_HY"
    ] = max(hd_df.index)

    # get positives years percentage and last negative year
    for pos_neg_years in [
        "lpa", "margembruta", "margemliquida",
    ]:
        pos_neg_years_df = hd_df[pos_neg_years].dropna() if pos_neg_years in hd_df else []
        if len(pos_neg_years_df) > 0:
            # % yers with positive values
            stock_infos_df.loc[
                stock_infos_df.index == stock_ticker,
                f"{pos_neg_years}_PY"
            ] = pos_neg_years_df.ge(0).sum() / len(pos_neg_years_df)
            # last negative year
            neg_years = pos_neg_years_df.loc[pos_neg_years_df.lt(0)]
            if len(neg_years) > 0:
                stock_infos_df.loc[
                    stock_infos_df.index == stock_ticker,
                    f"{pos_neg_years}_LNY"
                ] = neg_years.index[-1]

    # get average value
    for hist_avg_ind in [
        "lpa", "margembruta", "margemliquida", "dividaliquida_ebit",
    ]:
        hist_avg_df = hd_df[hist_avg_ind].dropna() if hist_avg_ind in hd_df else []
        if len(hist_avg_df) > 0:
            # trend
            stock_infos_df.loc[
                stock_infos_df.index == stock_ticker,
                f"{hist_avg_ind}_AV"
            ] = hist_avg_df.sum() / len(hist_avg_df)
    
    # get trend info
    for hist_trend_ind in [
        "lpa", "margembruta", "margemliquida", "dividaliquida_ebit",
    ]:
        hist_trend_df = hd_df[hist_trend_ind].dropna() if hist_trend_ind in hd_df else []
        if len(hist_trend_df) > 0:
            # trend
            stock_infos_df.loc[
                stock_infos_df.index == stock_ticker,
                f"{hist_trend_ind}_LR"
            ] = get_linear_regression_trend(hist_trend_df)

100%|██████████| 42/42 [00:34<00:00,  1.24it/s]


--------

In [13]:
filtered_data = stock_infos_df.join(
    stocks_general_data_df[[
        "PRECO",
        "P/L",
        "P/VP",
        "EV/EBIT",
        "DIVIDA LIQUIDA / EBIT",
        "MARGEM BRUTA",
        "MARG. LIQUIDA",
        "LIQ. CORRENTE",
        "CAGR RECEITAS 5 ANOS",
        "CAGR LUCROS 5 ANOS",
        "LIQUIDEZ MEDIA DIARIA",
        "VPA",
        "LPA",
    ]]
).join(
    extra_infos_df[[
        'Tickers Opções',
        'Vol Histórica',
    ]]
)

In [14]:
filtered_data = filtered_data[[
    'Ticker',
    'Nome da Empresa',
    'Setor de Atuação',
    'Nota dos Usuários',
    'Num. de Votos',
    'Segmento de Listagem',
    'Tag Along',
    'Free Float',
    'Listagem CVM',
    'Fundação',
    # Options
    'Tickers Opções',
    'Vol Histórica',
    # General Infos
    'LIQUIDEZ MEDIA DIARIA',
    'PRECO',
    'VPA',
    'P/L',
    'P/VP',
    'EV/EBIT',
    'LIQ. CORRENTE',
    'CAGR RECEITAS 5 ANOS',
    'CAGR LUCROS 5 ANOS',   
    # LPA
    'LPA',
    'lpa_AV',
    'lpa_LR',
    'lpa_PY',
    'lpa_LNY',
    # MARGEM BRUTA
    'MARGEM BRUTA', 
    'margembruta_AV',
    'margembruta_LR',
    'margembruta_PY',
    'margembruta_LNY',
    # MARG. LIQUIDA
    'MARG. LIQUIDA',
    'margemliquida_AV',
    'margemliquida_LR',
    'margemliquida_PY',
    'margemliquida_LNY',
    # DIVIDA LIQUIDA / EBIT
    'DIVIDA LIQUIDA / EBIT',
    'dividaliquida_ebit_AV',
    'dividaliquida_ebit_LR',
    # Years
    'Min_HY',
    'Max_HY', 
]]

-----

### Store result data

In [15]:
filtered_data.to_csv("../outputs/stocks_filter_v2.csv", sep=";")
filtered_data.to_excel("../outputs/stocks_filter_v2.xlsx")